  ## OUTLINE
  1. Load necessary packages
  2. Load data
  3. Data preprocessing
  4. Model designing
  5. Model modification

In [1]:
!pip install scikit-plot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install --upgrade keras
import pandas as pd
import numpy as np
import scikitplot
#import random
#import seaborn as sns
import keras
import os
from matplotlib import pyplot
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import warnings
from tensorflow.keras.models import Sequential
#from keras.callbacks import EarlyStopping
from keras import regularizers
from keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from tensorflow.keras.preprocessing.image import load_img
from keras.utils.vis_utils import plot_model
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization,MaxPooling2D,Activation,Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
warnings.simplefilter("ignore")
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.regularizers import l1, l2
import plotly.express as px
from matplotlib import pyplot as plt
import math
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
def pre_proc(data):
  train_pixels = data["pixels"]
  train_pixels = train_pixels.str.split(" ", expand = True)
  pixels = np.array(train_pixels)
  # train_pixels = np.float32(train_pixels)/255.
  #pixels = train_pixels.reshape((train_pixels.shape[0], 48, 48,1))
  return pixels 
  

In [4]:
def train_model(model, X_train, y_train, X_val, y_val):
    
    lr_rate = tf.keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate = 0.0001,
      decay_steps = 10000,
      decay_rate = 0.96)
    
    model.compile(optimizer=Adam(learning_rate= lr_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    
    checkpointer = [EarlyStopping(monitor = 'val_accuracy', verbose = 1, restore_best_weights=True,mode="max",patience = 10),
     ModelCheckpoint('best_model.h5',monitor="val_accuracy",verbose=1,
        save_best_only=True,mode="max")]
    
    
    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
                        validation_data=(X_val, y_val),callbacks = [checkpointer])
    
    loss, accuracy = model.evaluate(X_val, y_val)
    print("Test loss: {:.4f}".format(loss))
    print("Test accuracy: {:.4f}".format(accuracy))
    
    return history

In [5]:
def train_model_aug(model,X_train, y_train, X_val, y_val):
  
  lr_rate = tf.keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate = 0.0001,
      decay_steps = 10000,
      decay_rate = 0.96)
    
  model.compile(optimizer=Adam(learning_rate= lr_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    
  checkpointer = [EarlyStopping(monitor = 'val_accuracy', verbose = 1, restore_best_weights=True,mode="max",patience = 10),
     ModelCheckpoint('best_model.h5',monitor="val_accuracy",verbose=1,
        save_best_only=True,mode="max")]
  
  datagen.fit(X_train)
  
  history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
    verbose = 1,steps_per_epoch = int(len(X_train) / batch_size),
    epochs = epochs,validation_data = (X_val, y_val), callbacks = [checkpointer])
  return history

LOAD DATA

In [6]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [7]:
data = pd.read_csv('/content/drive/MyDrive/Advanced ML/my_emotion_train.csv')

In [8]:
data_test = pd.read_csv('/content/drive/MyDrive/Advanced ML/my_emotion_test.csv')


In [9]:
data.shape


(29000, 3)

check for missing data


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29000 entries, 0 to 28999
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       29000 non-null  int64 
 1   emotion  29000 non-null  int64 
 2   pixels   29000 non-null  object
dtypes: int64(2), object(1)
memory usage: 679.8+ KB


In [11]:
data.head()

,id,emotion,pixels
0,9415,6,29 16 18 18 18 20 19 18 17 17 17 18 17 18 17 1...
1,19109,3,126 154 167 181 188 194 195 194 196 195 198 20...
2,21523,2,169 220 218 208 184 144 72 73 143 183 203 210 ...
3,2076,3,60 64 72 80 83 83 80 82 89 106 114 125 125 127...
4,13957,3,174 148 121 97 78 70 62 57 54 54 42 58 40 64 1...


PREPROCESSING

#1. Asign label to the emotion

This is to make is easier to identify and relate with each emotion

In [12]:
EMOTION_LABELS  = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', "Surprise"]
fig = px.bar(x = EMOTION_LABELS,
             y = [list(data['emotion']).count(i) for i in np.unique(data['emotion'])] , 
             color = np.unique(data['emotion']) ,
             color_continuous_scale="blues") 
fig.update_xaxes(title="Emotions")
fig.update_yaxes(title = "Total Images")
fig.update_layout(showlegend = True,
    title = {
        'text': 'Emotion Distribution on Train  Data ',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show()

## 2. Shuffling the data

In order to minimise variance and ensure that the model generalise well to  unseen data points, the  data is being shuffled using sample() function and fraction was specified as 1 to shuffle the data by changing the rows.

In [13]:
data = data.sample(frac=1)

In [14]:
pixels = pre_proc(data)

In [15]:
scaler= StandardScaler()
pixels=scaler.fit_transform(pixels)

In [16]:
X_train, X_val, y_train, y_val = train_test_split(pixels,data.emotion, test_size=0.2, shuffle=False)
y_train,y_val = to_categorical(y_train),to_categorical(y_val)

In [17]:
 from keras.preprocessing.image import ImageDataGenerator

In [18]:
datagen = ImageDataGenerator(
featurewise_center=True,
featurewise_std_normalization=True,
rotation_range=20,
width_shift_range=0.2,
height_shift_range=0.2,
horizontal_flip=True,
validation_split=0.2)

In [19]:
regularizer = tf.keras.regularizers.L1L2(l1=0.01,l2=0.01)
batch_size,epochs = 32,100

In [20]:
X_train,X_val= X_train.reshape((X_train.shape[0], 48, 48,1)),X_val.reshape((X_val.shape[0], 48, 48,1))

# Baseline ML model

In [21]:
model_baseline = keras.models.Sequential([
        keras.layers.Flatten(input_shape=(48, 48,1)),
        keras.layers.Dense(300, activation="relu"),
        keras.layers.Dense(7, activation="softmax")
])

Without Augumented data

In [ ]:
base_model = train_model(model_baseline,X_train, y_train, X_val, y_val)


With Augumented data

In [ ]:
base_model_aug = train_model_aug(model_baseline,X_train, y_train, X_val, y_val)

##CNN model

CNN model has many layers with different units such as Convolutional layer, Max pooling layer, Batch normalization and Drop out layer to regularize the model.




In [24]:
model_cnn = tf.keras.models.Sequential()
model_cnn.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(48, 48,1)))
model_cnn.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPool2D(pool_size=(2, 2)))
model_cnn.add(Dropout(0.25))

model_cnn.add(Conv2D(128,(5,5), padding='same', activation='relu'))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPool2D(pool_size=(2, 2)))
model_cnn.add(Dropout(0.25))
    
model_cnn.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizer))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPool2D(pool_size=(2, 2)))
model_cnn.add(Dropout(0.25))
model_cnn.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizer))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPool2D(pool_size=(2, 2)))
model_cnn.add(Dropout(0.25))

model_cnn.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizer))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPool2D(pool_size=(2, 2)))
model_cnn.add(Dropout(0.25))

model_cnn.add(Flatten()) 
model_cnn.add(Dense(256,activation = 'relu'))
model_cnn.add(BatchNormalization())
model_cnn.add(Dropout(0.25))
    
model_cnn.add(Dense(512,activation = 'relu'))
model_cnn.add(BatchNormalization())
model_cnn.add(Dropout(0.25))

model_cnn.add(Dense(7, activation='softmax'))

Without Augumentation

In [ ]:
cnn_model = train_model(model_cnn, X_train, y_train, X_val, y_val)

With Augumentation

In [ ]:
cnn_model_aug = train_model_aug(model_cnn, X_train, y_train, X_val, y_val)

# ANN Model

In [27]:
model_ann = keras.models.Sequential([
        keras.layers.Flatten(input_shape=(48,48,1)),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(300, activation="relu", kernel_initializer="glorot_normal",kernel_regularizer=regularizer),
        keras.layers.Dense(300, activation="relu", kernel_initializer="glorot_normal"),
        keras.layers.Dense(100, activation="relu", kernel_initializer="glorot_normal",kernel_regularizer=regularizer),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(30, activation="relu", kernel_initializer="glorot_normal"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(7, activation="softmax")
])

Without Augumentation

In [ ]:
ann_model = train_model(model_ann, X_train, y_train, X_val, y_val)

With Augumentation

In [ ]:
ann_model_aug = train_model_aug(model_ann, X_train, y_train, X_val, y_val)

RESTNET 

In [30]:
class ResidualUnit(keras.layers.Layer):
  def __init__(self, filters, strides=1, activation="relu", **kwargs):
    super().__init__(**kwargs)
    self.activation = keras.activations.get(activation)
    self.main_layers = [
      keras.layers.Conv2D(filters, 3, strides=strides, padding="same", use_bias=False),
      keras.layers.BatchNormalization(), 
      self.activation, 
      keras.layers.Conv2D(filters, 3, strides=1, padding="same", use_bias=False), 
      keras.layers.BatchNormalization()
    ]

    self.skip_layers = [
      keras.layers.Conv2D(filters, 1, strides=strides, padding="same", use_bias=False), 
      keras.layers.BatchNormalization()
    ]

  def call(self, inputs): 
    Z = inputs
    for layer in self.main_layers: 
      Z = layer(Z)
    skip_Z = inputs
    for layer in self.skip_layers:
      skip_Z = layer(skip_Z)
    return self.activation(Z + skip_Z)

In [ ]:
from keras.layers import Conv2D
model_rest = keras.models.Sequential()
model_rest.add(keras.layers.Conv2D(64, 7, strides=1, input_shape=[48, 48, 1], padding="same", use_bias=False)) 
model_rest.add(keras.layers.BatchNormalization())
model_rest.add(keras.layers.Activation("relu")) 
model_rest.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same")) 
prev_filters = 64 
for filters in [64]*3+[128]*4+[256]*6+[512]*3:
  strides = 1 if filters == prev_filters else 2 
  model_rest.add(ResidualUnit(filters, strides=strides)) 
  prev_filters = filters
model_rest.add(keras.layers.GlobalAvgPool2D()) 
model_rest.add(keras.layers.Flatten()) 
model_rest.add(keras.layers.Dense(7, activation="softmax"))
model_rest.summary()

### Without Augumentation

In [ ]:
resnet_model = train_model(model_rest, X_train, y_train, X_val, y_val)

### With Augumentation

In [ ]:
resnet_model_aug = train_model_aug(model_rest, X_train, y_train, X_val, y_val)

# Preparing the test data set

In [34]:
data_test = data_test.sample(frac=1)
pixel_test = pre_proc(data_test)
X_test=scaler.fit_transform(pixel_test)
X_test= X_test.reshape((X_test.shape[0], 48, 48,1))

# Training model Summary

In [35]:
models = [('base_model', base_model),('base_model_aug', base_model_aug),
         ('ann_model', ann_model), ('ann_model_aug', ann_model_aug),
          ('cnn_model', cnn_model), ('cnn_model_aug', cnn_model_aug),
          ('resnet_model', resnet_model),('resnet_model_aug', resnet_model_aug)]
          
outputs = [model_baseline,model_baseline,model_ann,model_ann,
           model_cnn,model_cnn,model_rest,model_rest]

results = []
for model_name, model in models:
    val_acc = round(model.history['val_accuracy'][-1], 2)
    model_idx = models.index((model_name, model))
    x = np.round(outputs[model_idx].predict(X_val))
    mean = round(np.mean(x),2)
    std = round(np.std(x),2)
    model_results = {'Model': model_name, 'Accuracy': val_acc, 'Mean': mean, 'STD': std}
    results.append(model_results)

df = pd.DataFrame(results, columns=['Model', 'Accuracy', 'Mean', 'STD'])
print(df)

182/182 [==============================] - 1s 8ms/step
              Model  Accuracy  Mean   STD
0        base_model      0.42  0.09  0.29
1    base_model_aug      0.33  0.09  0.29
2         ann_model      0.25  0.00  0.00
3     ann_model_aug      0.25  0.00  0.00
4         cnn_model      0.55  0.06  0.25
5     cnn_model_aug      0.56  0.06  0.25
6      resnet_model      0.50  0.11  0.31
7  resnet_model_aug      0.63  0.11  0.31


# Test Model Summary

In [38]:
models = {'Base Model':[0.55,0.53],'CNN':[0.6,0.59],'RNN': [0.59,0.59]}

results = []
for model_name, model_scores in models.items():
    private_score = model_scores[0]
    public_score = model_scores[1]
    model_results = {'Model': model_name, 'Private score': private_score,'Public score': public_score}
    results.append(model_results)

df = pd.DataFrame(results, columns=['Model', 'Private score','Public score'])
print(df)

        Model  Private score  Public score
0  Base Model           0.55          0.53
1         CNN           0.60          0.59
2         RNN           0.59          0.59
